# Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [4]:
# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")


### VectorDB Instance

In [5]:
# Instantiate your ChromaDB Client
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [6]:
# OpenAI embedding function
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY
)

In [7]:
# Create a collection
collection = chroma_client.create_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

### Add documents

In [8]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [10]:
# Verify the documents were added
print(f"Collection count: {collection.count()}")
print(f"Collection name: {collection.name}")

# Get a sample document to verify
sample = collection.get(limit=1, include=["documents", "metadatas"])
if sample["ids"]:
    print(f"\nSample document ID: {sample['ids'][0]}")
    print(f"Sample content: {sample['documents'][0]}")
    print(f"Sample metadata: {sample['metadatas'][0]}")

Collection count: 15
Collection name: udaplay

Sample document ID: 001
Sample content: [PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.
Sample metadata: {'Name': 'Gran Turismo', 'Genre': 'Racing', 'Platform': 'PlayStation 1', 'Publisher': 'Sony Computer Entertainment', 'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.', 'YearOfRelease': 1997}


### Semantic Search Demo

In [11]:
# Demonstrate semantic search capabilities
def search_games(query, n_results=3):
    """
    Search for games using semantic similarity
    """
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    print(f"🔍 Search Query: '{query}'")
    print(f"📊 Found {len(results['ids'][0])} results:\n")
    
    for i, (doc_id, document, metadata, distance) in enumerate(zip(
        results['ids'][0], 
        results['documents'][0], 
        results['metadatas'][0], 
        results['distances'][0]
    )):
        print(f"🎮 Result {i+1} (Similarity: {1-distance:.3f})")
        print(f"   Game: {metadata['Name']}")
        print(f"   Platform: {metadata['Platform']}")
        print(f"   Genre: {metadata['Genre']}")
        print(f"   Year: {metadata['YearOfRelease']}")
        print(f"   Description: {metadata['Description']}")
        print(f"   Document ID: {doc_id}")
        print("-" * 80)

# Test different types of semantic searches
print("=" * 80)
search_games("racing games with realistic driving simulation")

print("\n" + "=" * 80)
search_games("retro console games from the 90s")

print("\n" + "=" * 80)
search_games("action adventure games with exploration")

🔍 Search Query: 'racing games with realistic driving simulation'
📊 Found 3 results:

🎮 Result 1 (Similarity: 0.760)
   Game: Gran Turismo
   Platform: PlayStation 1
   Genre: Racing
   Year: 1997
   Description: A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.
   Document ID: 001
--------------------------------------------------------------------------------
🎮 Result 2 (Similarity: 0.750)
   Game: Gran Turismo 5
   Platform: PlayStation 3
   Genre: Racing
   Year: 2010
   Description: A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.
   Document ID: 003
--------------------------------------------------------------------------------
🎮 Result 3 (Similarity: 0.602)
   Game: Mario Kart 8 Deluxe
   Platform: Nintendo Switch
   Genre: Racing
   Year: 2017
   Description: An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay me